## Librerias

In [1]:
!pip install pyfim

In [2]:
!pip install --upgrade mlxtend

In [3]:
!pip install --upgrade numpy
!pip install --upgrade mlxtend


In [4]:
from fim import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import association_rules

## Dataset

In [5]:
df = pd.read_csv("clean_dataset.csv")
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1


In [6]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(690, 16)

Convertir 0 a 1 para poder visualizar de manera clara

Prior Default relacion inversa (basandose en comentario en kaggle)

"Given the 0.7 correlation between PriorDefault and Approved columns, I think PriorDefault has a reversed meaning than the meaning given in the dataset now. Using the given meaning, if you default previously, you will have a higher chance of getting the credit card approval, which sounds crazy."

In [7]:
df.replace({'Gender': {0: 'Female', 1: 'Male'},
            'Married': {0: 'No Casado', 1: 'Casado'},
            'BankCustomer': {0: 'No Cliente', 1: 'Cliente'},
            'PriorDefault': {0: ' Prestamo atrasado', 1: 'No Prestamo atrasado'},
            'Employed': {0: 'No Trabaja', 1: 'Trabaja'},
            'DriversLicense': {0: 'No Licencia de conducir', 1: 'Licencia de conducir'}}, inplace=True)
            #'Approved': {0: 'Denegado', 1: 'Aprobado'}}, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.f

Quitar objetivo (Approved) y convertir a string

In [8]:
X = df.drop('Approved', axis=1)
X = X.astype(str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## fpgrowth

Convertir a Transactions

In [9]:
transactions = X.values.tolist()

# Inicializar y ajustar el modelo FP-Growth
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_transformed = pd.DataFrame(te_ary, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# Agregar la columna 'Approved' al DataFrame transformado
df_transformed['Approved'] = df['Approved']

# Aplicar el algoritmo FP-Growth
frequent_itemsets = fpgrowth(df_transformed, min_support=0.3, use_colnames=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [11]:
print(frequent_itemsets)

      support                                      itemsets
0    0.905797                                     (ByBirth)
1    0.763768                                     (Cliente)
2    0.760870                                      (Casado)
3    0.742029                                           (0)
4    0.695652                                        (Male)
..        ...                                           ...
147  0.334783              (0,  Prestamo atrasado, ByBirth)
148  0.379710           (0,  Prestamo atrasado, No Trabaja)
149  0.318841     (No Trabaja,  Prestamo atrasado, ByBirth)
150  0.318841  (0,  Prestamo atrasado, ByBirth, No Trabaja)
151  0.331884         (Cliente,  Prestamo atrasado, Casado)

[152 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Reglas de asociación

In [12]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
rules_approved = rules[rules['consequents'].apply(lambda x: 'Approved' in x)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
selected_columns = ['consequents', 'antecedents', 'support', 'confidence', 'lift']
#print(rules_approved.sort_values(by='support', ascending=False).head(10))

show = rules_approved.sort_values(by='support', ascending=False).head(10)

show = show[selected_columns]
print(show)

                          consequents                      antecedents  \
410                        (Approved)                        (ByBirth)   
412                        (Approved)           (No Prestamo atrasado)   
420                        (Approved)  (No Prestamo atrasado, ByBirth)   
423               (ByBirth, Approved)           (No Prestamo atrasado)   
424  (No Prestamo atrasado, Approved)                        (ByBirth)   
414                        (Approved)                        (Cliente)   
452                        (Approved)                (Cliente, Casado)   
456               (Cliente, Approved)                         (Casado)   
455                (Casado, Approved)                        (Cliente)   
416                        (Approved)                         (Casado)   

      support  confidence      lift  
410  0.415942    0.459200  1.032078  
412  0.411594    0.786704  1.768161  
420  0.397101    0.805882  1.811267  
423  0.397101    0.759003  1.8247

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Las caracteristicas mas importantes para poder tener una tarjeta de credito aprovada es que tengas nacionalidad por nacimiento y que no tengas prestamos atrasados. Estoas caracteristicas tienen un support mayor al 40% ademas de tener una asociación positiva evidenciada por tener un lift mayor que 1. Finalmente es mas comun que si no tienes prestamos atrasados te acepten la tarjeta de credito debido a la alta confianza.

El tercer antecedente mas comun para obtener una tarjeta aprobada es la combinacion de no tener prestamos atrasados y de tener nacionalidad por nacimiento. Tener estas 2 caracteristicas juntas te da una mayor posibilidad de que te acepten la tarjeta envidenciado en el alto lift.


Las siguientes caracteristicas que mas influyen son si ya eres cliente del banco y si estas casado estas tienen un support mayor al 37%. Ademas tener estas caracteristicas tambien aumenta la posibilidad de que te acepten tu tarjeta, esto se puede ver por el lift mayor que 1.